In [1]:
import numpy as np
import scipy.stats as ss
import ipywidgets as wd
import yaml
import time

In [2]:
sleeptime = 1  # delay (seconds) between messages.

In [3]:
def beta_dist(min_value=0, max_value=1, a=1, b=1):
    return ss.beta(loc=min_value, scale=(max_value-min_value), a=a, b=b)

def auto_fill(height='auto', width='auto', label=''):
    return wd.Button(description=label, layout=auto_lay(width,height))

def auto_lay(width='auto', height='auto'):
    return wd.Layout(height=height, width=width)


In [4]:
def mid_preset(min_val, max_val, confidence):
    """
    Returns beta distribution parameters (symmetric)
    """
    loc = min_val
    scale = max_val - min_val
    a = b = np.maximum(np.minimum(confidence+1,3),0.1)**2
    return {'a': a, 'b': b, 'loc': loc, 'scale': scale}

def high_preset(min_val, max_val, confidence):
    """
    Returns beta distribution parameters (left skewed)
    """
    loc = min_val
    scale = max_val - min_val
    b = np.minimum(2.5,confidence+1.5)
    b = np.maximum(b,0.1)
    a = 6 - b 
    return {'a': a, 'b': b, 'loc': loc, 'scale': scale}

def low_preset(min_val, max_val, confidence):
    """
    Returns beta distribution parameters (right skewed)
    """
    loc = min_val
    scale = max_val - min_val
    a = np.minimum(confidence+1.5,2.5)
    a = np.maximum(a,0.1)
    b = 6 - a 
    return {'a': a, 'b': b, 'loc': loc, 'scale': scale}

In [18]:
def make_preset_grid():
    """
    Return grid layout (2x3) with three buttons and a slider,
    for preset definitions.
    """
    presets = wd.GridspecLayout(2,3)
    btn_left = wd.Button(description='left', layout=auto_lay())
    btn_cent= wd.Button(description='center', layout=auto_lay())
    btn_right = wd.Button(description='right', layout=auto_lay())
    opt_slide = wd.FloatSlider(description='shape', value=0, min=0,max=1,step=0.1,
                               layout=auto_lay())
    presets[0,0] = btn_left
    presets[0,1] = btn_cent
    presets[0,2] = btn_right
    presets[1,:] = opt_slide
    return presets, [btn_left, btn_cent, btn_right], opt_slide

In [6]:
def make_db_setter(DB):
    param_setter = wd.GridspecLayout(2,3)#, layout=wd.Layout(height='auto', width='auto'))
    param_list = list(DB.keys())
    param_dropdown = wd.Dropdown(options=param_list, description='Dim',layout=wd.Layout(height='100%', width='auto'))
    param_setter[0,0:2] = param_dropdown
    param_text = wd.Text(description = 'Name', layout=wd.Layout(height='100%', width='100%'))
    param_setter[1,0:2] = param_text

    add_button = wd.Button(description='add', layout=wd.Layout(height='100%', width='80%'))
    rem_button  = wd.Button(description='rem', layout=wd.Layout(height='100%', width='20%'))

    param_setter[1,2] = wd.HBox([rem_button, add_button])
    info = wd.Label()
    param_setter[0,2] = info
    return param_setter, [info, param_dropdown, param_text, add_button, rem_button]

In [27]:
beta_setter = wd.GridspecLayout(2,2, layout=wd.Layout(height='auto', width='auto'))
min_txt = wd.FloatText(value=0, description="MIN", layout=wd.Layout(height='auto', width='auto'))
max_txt = wd.FloatText(value=1, description="MAX", layout=wd.Layout(height='auto', width='auto'))
beta_setter[0,0] = min_txt
beta_setter[0,1] = max_txt
a = wd.FloatSlider(value=1, min=0, max=10, description='a', layout=wd.Layout(height='auto', width='auto'))
b = wd.FloatSlider(value=1, min=0, max=10, description='b', layout=wd.Layout(height='auto', width='auto'))
beta_setter[1,0] = a
beta_setter[1,1] = b

In [36]:
# Initiate (Empty) Database

DB = {}
db_name = 'db.yaml'
# with open(db_name, 'w') as db:
#     yaml.dump(DB, db)

In [37]:
with open(db_name, 'r') as db:
        DB = yaml.load(db, Loader=yaml.FullLoader)

In [38]:
param_setter, widgets = make_db_setter(DB)
info, param_dropdown, param_text, add_button, rem_button = widgets

presets, buttons, opt_slide = make_preset_grid()
btn_left, btn_cent, btn_right = buttons

In [39]:
def get_states(*args):
    """
    Read states from sliders.
    """
    if min_txt.value > max_txt.value:
        info.value = "Min > Max. Switching..."
        time.sleep(sleeptime)
        info.value = ""
        mn, mx = np.copy(max_txt.value), np.copy(min_txt.value)
        max_txt.value = mx
        min_txt.value = mn
    vals = [min_txt.value, max_txt.value]
    return {'loc': vals[0], 'scale': vals[1] - vals[0], 'a': a.value, 'b': b.value}

def update_state(state):
    """
    Set states on sliders.
    """
    max_txt.value = state['loc'] + state['scale']
    min_txt.value = state['loc']
    a.value = state['a']
    b.value = state['b']

def add_key(*args):
    with open(db_name, 'r') as db:
        DB = yaml.load(db, Loader=yaml.FullLoader)
    key_list = DB.keys() # grab keys from back-end
    
    txt = param_text.value.replace(' ', '_')
    if len(txt) > 0:
        states = get_states(args)
        
        # if adding a new entry, copy over existing info.
        if txt not in key_list: 
            state_of_operation = "added"
        else:  # overwriting
            state_of_operation = "overwritten"

        DB[txt] = get_states(args)
        info.value = txt + " : " + " has been %s."%state_of_operation
        time.sleep(sleeptime)
#         info.value = " "
#         # clean up list
#         if 'default' in key_list:
#             DB.pop('default')
        with open(db_name, 'w') as db:
            yaml.dump(DB, db)
        param_dropdown.options = list(DB.keys())
        param_dropdown.value = txt
    else:
        info.value = "Nothing was entered."


def lookup_key(*args):
    with open(db_name, 'r') as db:
        DB = yaml.load(db, Loader=yaml.FullLoader)
    try:
        state = DB[param_dropdown.value]
        # TODO: Set states
#         param_text.value = param_dropdown.value
        param_text.value = "" # clear
        max_txt.value = state['loc'] + state['scale']
        min_txt.value = state['loc']
        a.value = state['a']
        b.value = state['b']
        info.value = "Loaded database state."
    except KeyError:
        info.value = "No matching entry."
    except:  # attempting to set empty string
        pass
    time.sleep(sleeptime)
#     info.value = ""


def remove_var(*args):
    with open(db_name, 'r') as db:
        DB = yaml.load(db, Loader=yaml.FullLoader)
    txt = param_text.value.replace(' ', '_')
    if len(txt) > 0:
        if txt in list(DB.keys()): # if key exists, remove it
            info.value = "%s was removed."%param_dropdown.value
            DB.pop(txt)
            with open(db_name, 'w') as db:
                    yaml.dump(DB, db)
            time.sleep(sleeptime)
            param_dropdown.options = list(DB.keys())
        else:  # otherwise raise error
            info.value = "Entry not in database."
            time.sleep(sleeptime)
#         info.value = ""
        # update dropdown
        if len(param_dropdown.value) == 0:  # empty entry
            pass
        else:
            param_text.value = param_dropdown.value
    else:
        info.value = "Please enter some text."
        time.sleep(sleeptime)
#         info.value = ""
        
        
def preset_btn_left(*args):
    state = low_preset(min_txt.value,max_txt.value,opt_slide.value)
    update_state(state)
    
def preset_btn_cent(*args):
    state = mid_preset(min_txt.value,max_txt.value,opt_slide.value)
    update_state(state)
    
def preset_btn_right(*args):
    state = high_preset(min_txt.value,max_txt.value,opt_slide.value)
    update_state(state)
    
btn_left.on_click(preset_btn_left)
btn_cent.on_click(preset_btn_cent)
btn_right.on_click(preset_btn_right)

param_dropdown.observe(lookup_key, 'value')
rem_button.on_click(remove_var)
add_button.on_click(add_key)
# lookup_key()

In [40]:
# head = wd.Label("Potential Value",layout=wd.Layout(height='auto', width='auto'))
# foot = wd.IntSlider(layout=wd.Layout(height='auto', width='auto'))
# left = None
# cent = auto_fill()
# right = None

In [41]:
# APP = wd.AppLayout(header=None, 
#              footer=None,
#              left_sidebar=param_setter,
#              right_sidebar=beta_setter,
#              center=None)

In [42]:
# display(APP)

In [43]:
G = wd.GridspecLayout(10,2)
G[0,:] = param_setter
# G[1,0] = auto_fill(height='10%')
G[1,:] = beta_setter
# G[0:2,:] = auto_fill(label="SETTINGS")
G[2:4,:] = presets
G[4:9,0] = auto_fill(label='PLOT')
G[4:9,1] = auto_fill(label='PLOT')
# G[7,:] = beta_setter
# G[8,:] = param_setter
G[9:10,0:] = auto_fill(label='RUN')
# G[,2] = wd.Button(layout=wd.Layout(height='auto', width='50%'))
# G[:,2] = wd.Button(layout=wd.Layout(height='auto', width='auto'))

In [44]:
display(G)

GridspecLayout(children=(GridspecLayout(children=(Dropdown(description='Dim', layout=Layout(grid_area='widget0…

In [45]:
with open(db_name, 'r') as db:
        DB = yaml.load(db, Loader=yaml.FullLoader)
DB

{'one': {'a': 1.5, 'b': 4.5, 'loc': 0.0, 'scale': 1.0},
 'two': {'a': 4.5, 'b': 1.5, 'loc': 0.0, 'scale': 1.0}}

In [ ]:
mid_preset(0,1,1)

In [ ]:
low_preset(0,1,10)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# %matplotlib widget 

In [ ]:
def compare_plots(val):
    x = np.linspace(0,1,100)
    plt.plot(x, ss.beta(**low_preset(0,1,val)).pdf(x))
    plt.plot(x, ss.beta(**mid_preset(0,1,val)).pdf(x))
    plt.plot(x, ss.beta(**high_preset(0,1,val)).pdf(x))
    plt.show()

In [ ]:
wd.interactive(compare_plots, val=wd.FloatSlider(value=2,min=0,max=1,step=0.1))

In [ ]:
cc = wd.Controller()
cc